In [ ]:
# importing dependencies

import time,random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
import numpy as np
import time, random, concurrent.futures
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
entire_data = [] # loading the data

with open('all_cars.json','r') as f:
        data = json.load(f)
        entire_data.extend(list(data.values())[0])

In [ ]:
def get_driver():
    """
    This function do not expect any input, and it returns a webdriver object of chrome browser with few settings in options params
    """

    options = webdriver.ChromeOptions()

    # It operates without opening a visible browser window.
    options.add_argument("--headless=new") 

    # Some servers (like Google Colab, Docker, or cloud machines) block Chrome from starting with its default safety settings. 
    # Adding this option makes sure Chrome doesn’t crash and your scraper keeps running.
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    return webdriver.Chrome(options=options)

def scrape_page(link):
    """
    This function take a link as input and extract each and every detail of car of that link a dict that holds result of perticular car
    """
    driver = None

    # --- init with np.nan to prevent data shifting issue ---
    data = {
        "image_list": np.nan,
        "new_used": np.nan,
        "car_name": np.nan,
        "mileage": np.nan,
        "price": np.nan,
        "price_drop": np.nan,
        "deal_type": np.nan,
        "key_specs": np.nan,
        "basics": np.nan,
        "features": np.nan,
        "other_features": np.nan,
        "vehicle_history": np.nan,
        "seller_name": np.nan,
        "seller_rating": np.nan,
        "people_count_seller_rating": np.nan,
        "seller_address": np.nan,
        "seller_site": np.nan,
        "car_rating": np.nan,
        "people_count_car_rating": np.nan,
    }

    try:

        # --- Creating a Driver Object ---
        driver = get_driver()
        driver.get(link)
        time.sleep(random.choice([2, 3])) # time to get load
        soup = BeautifulSoup(driver.page_source, "html.parser") 

        # --- Extract fields ---

        # IMAGES
        try:
            imgs = []
            selected_img = soup.find("img", attrs={"class":"swipe-main-image image-index-0"})
            if selected_img:
                imgs.append(selected_img.get("src"))
            for image in soup.find_all("img", attrs={"class":"row-pic"}):
                imgs.append(image.get("src"))
            data["image_list"] = imgs if imgs else np.nan
        except:
            pass

        # NEW OR USED
        try:
            data["new_used"] = soup.find("p","new-used").text
        except:
            pass

        # CAR NAME
        try: 
            data["car_name"] = soup.find("h1","listing-title").text
        except: 
            pass

        # MILEAGE
        try: 
            data["mileage"] = soup.find("p","listing-mileage").text
        except: 
            pass

        # PRICE
        try: 
            data["price"] = soup.find("span", attrs={"data-qa":"primary-price"}).text
        except: 
            pass

        # PRICE_DROP
        try:
            data["price_drop"] = soup.find("span", attrs={"data-qa":"price-drop"}).text
        except:
            pass

        # DEAL TYPE
        try: 
            data["deal_type"] = soup.find("span", attrs={"data-qa":"price-badge-text"}).text
        except: 
            pass

        # --- Key Specs ---
        keyspec_dic = {}
        try:
            for i in soup.find("div",id="key-specs-container").find("dl",id="key-spec-details-container").find_all("div","key-spec"):
                keyspec_dic[i.find("dd").text] = i.find("dt").text
        except:
            pass
        try:
            for i in soup.find("dl",id="insight-details-container").find_all("div","key-spec"):
                keyspec_dic[i.find("dd").text] = i.find("dt").text
        except:
            pass
        if keyspec_dic:
            data["key_specs"] = keyspec_dic

        # --- Basics ---
        try:
            basics = {}
            for dt,dd in zip(
                soup.find("section","sds-page-section basics-section").find_all("dt"),
                soup.find("section","sds-page-section basics-section").find_all("dd")
            ):
                basics[dt.text] = dd.text
            data["basics"] = basics
        except: 
            pass

        # --- Features ---
        try:
            features_dic = {}
            for dt,dd in zip(
                soup.find("section","sds-page-section features-section").find_all("dt"),
                soup.find("section","sds-page-section features-section").find("dl","fancy-description-list").find_all("dd")
            ):
                features_dic[dt.text] = dd.text
            data["features"] = features_dic
        except: 
            pass

        # Other Features
        try: 
            data["other_features"] = [li.text for li in soup.find("div","sds-modal sds-modal-one view-all-features-container").find_all("li")]
        except: 
            pass

        # Vehicle History
        try:
            history_dic = {}
            for dt,dd in zip(
                soup.find("section","sds-page-section vehicle-history-section").find_all("dt"),
                soup.find("section","sds-page-section vehicle-history-section").find_all("dd")
            ):
                history_dic[dt.text] = dd.text
            data["vehicle_history"] = history_dic
        except: 
            pass

        # Seller Name
        try: 
            data["seller_name"] = soup.find("h3","spark-heading-5 heading seller-name").text
        except: 
            pass

        # Seller Rating
        try: 
            data["seller_rating"] = soup.find("spark-rating").get("rating")
        except: 
            pass

        # People count that rated seller
        try: 
            data["people_count_seller_rating"] = soup.find("spark-rating").text
        except: 
            pass

        # Seller Address
        try: 
            data["seller_address"] = soup.find("div","dealer-address").text
        except: 
            pass

        # Seller Site
        try: 
            data["seller_site"] = soup.find("a",{"data-linkname":"dealer-external-site"}).get("href")
        except: 
            pass

        # Car rating
        temp = None
        try:
            temp = soup.find("div","vehicle-reviews generated-consumer-review-shown").find("spark-rating",attrs={"size":"overview"}).get("rating")
        except: pass
        if not temp:
            try: temp = soup.find("div","vehicle-reviews").find("spark-rating",attrs={"size":"overview"}).get("rating")
            except: pass
        data["car_rating"] = temp if temp else np.nan

        # People count car rating
        temp = None
        try:
            temp = soup.find("div","vehicle-reviews generated-consumer-review-shown").find("spark-rating",attrs={"size":"overview"}).find("a").text
        except: pass
        if not temp:
            try: temp = soup.find("div","vehicle-reviews").find("spark-rating",attrs={"size":"overview"}).find("a").text
            except: pass
        data["people_count_car_rating"] = temp if temp else np.nan

    except Exception as e:
        print(f"Error scraping {link}: {e}")
    finally:
        if driver:
            driver.quit()
    return data

results = [] # this list will hold all dicts
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor: # At most 4 tasks will run at the same time (in parallel, using threads).
    for idx, result in enumerate(executor.map(scrape_page, entire_data[322000:323000])):
        results.append(result)
        print(f"{idx+1}/{len(entire_data)} done")

In [ ]:
df = pd.DataFrame(results)

df.to_csv('all-cars.csv') # exporting the results in csv file format

In [ ]:
# Again, instead of doing this entire work alone on my single local machine, I used colabs from my 3 different emails to prevent getting 
# blocked by server and executing the work faster, but because there are too many links, slow response from server I also used the kaggle notebook
# hence instead of using just 1 local machine, I used (5 (kaggle_server) + 3 colab server) * 3 mails = 32 server in total

# I scraped the data and stored each file in final_uncleaned_data folder , which will be further used for cleaning, but to reduce the size
# I converted the data into parquet format (stores data into binary format, and divided entire data into 5 diff chunks in total)

import pandas as pd
import glob
import os

folder_path = "C:/Users/aksha/OneDrive/Desktop/CARS-FINAL_YEAR_PROJECT/final_uncleaned_data"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

print(df.shape)

df = df.sample(335956) # shuffled the data

n_splits = 5 
rows_per_file = len(df) // n_splits  

for i in range(n_splits):
    start = i * rows_per_file
    end = (i + 1) * rows_per_file if i < n_splits - 1 else len(df)

    df_chunk = df.iloc[start:end] 

    print(df_chunk.shape)

    df_chunk.to_parquet(f"data_part_{i+1}.parquet", index=False) # exporting to chunks